In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy ADK Agents to Vertex AI Agent Engine Using Inline Source

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fagent_engine%2Fadk_inline_source_deployment.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/agent_engine/adk_inline_source_deployment.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates how to deploy [Google Agent Development Kit (ADK)](https://google.github.io/adk-docs/) agents to [Vertex AI Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) using **inline source deployment**.

I built this pattern while developing a multi-agent system with ADK. The approach is also featured in Google's [Agent Starter Pack community showcase](https://github.com/GoogleCloudPlatform/agent-starter-pack#community-showcase).

Inline source deployment sends your agent's Python source code directly to Agent Engine, avoiding serialization issues that can occur with the default pickle-based deployment.

### Why Inline Source Deployment?

| Approach | How it works | Best for |
|----------|--------------|----------|
| Default (pickle) | Serializes agent object | Simple agents, quick prototypes |
| **Inline source** | Sends Python source directly | Complex agents, custom dependencies |

Inline source deployment is useful when:
- Your agent uses custom classes that don't serialize well
- You need precise control over the deployment environment
- You want to avoid pickle compatibility issues across Python versions

### Objectives

- Create an ADK agent with tools
- Deploy to Agent Engine using inline source
- Test the deployed agent
- Clean up resources

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI Agent Engine
* Vertex AI (Gemini API)

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Get started

### Install required packages

In [ ]:
# Define requirements once for reuse
REQUIREMENTS = [
    "google-cloud-aiplatform[adk,agent_engines]>=1.88.0",
    "google-adk>=1.18.0",
]

!pip install --upgrade --quiet {" ".join(REQUIREMENTS)}

### Restart runtime

After installing packages, restart the runtime to ensure the new packages are loaded.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticate your notebook environment (Colab only)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1"  # @param {type:"string"}

STAGING_BUCKET = f"gs://{PROJECT_ID}-agent-engine-staging"  # @param {type:"string"}

### Create staging bucket

Agent Engine requires a staging bucket to store deployment artifacts.

In [ ]:
!gsutil mb -l {LOCATION} -p {PROJECT_ID} {STAGING_BUCKET} 2>/dev/null || echo "Bucket already exists"

### Initialize Vertex AI

In [ ]:
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

## Create an ADK Agent

First, let's create a simple ADK agent with a tool. This agent can answer questions and get the current time.

In [ ]:
import datetime
from zoneinfo import ZoneInfo, ZoneInfoNotFoundError

from google.adk.agents import LlmAgent


def get_current_time(timezone: str = "UTC") -> str:
    """Get the current time in the specified timezone.

    Args:
        timezone: IANA timezone name (e.g., 'America/New_York', 'UTC')

    Returns:
        Current time as a formatted string
    """
    try:
        tz = ZoneInfo(timezone)
        now = datetime.datetime.now(tz)
        return f"The current time in {timezone} is {now.strftime('%Y-%m-%d %H:%M:%S %Z')}"
    except ZoneInfoNotFoundError:
        return f"Error: Timezone '{timezone}' not found. Please use a valid IANA timezone name (e.g., 'America/New_York', 'UTC')."
    except Exception as e:
        return f"An unexpected error occurred while getting time for timezone {timezone}: {e}"


# Create the agent
agent = LlmAgent(
    name="time_agent",
    model="gemini-1.5-flash-001",
    instruction="You are a helpful assistant. Use the get_current_time tool when asked about the time.",
    tools=[get_current_time],
)

print(f"Created agent: {agent.name}")

### Test the agent locally

Before deploying, let's test the agent locally to make sure it works.

In [ ]:
from google.adk import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create a local runner
runner = Runner(
    agent=agent,
    app_name="test_app",
    session_service=InMemorySessionService(),
)

# Create a session and test
session = runner.session_service.create_session(
    app_name="test_app",
    user_id="test_user",
)

# Send a test message
response = runner.run(
    user_id="test_user",
    session_id=session.id,
    new_message=types.Content(
        role="user",
        parts=[types.Part(text="What time is it in New York?")],
    ),
)

# Print the response
for event in response:
    if hasattr(event, "content") and event.content:
        for part in event.content.parts:
            if hasattr(part, "text") and part.text:
                print(part.text)

## Deploy Using Inline Source

Now let's deploy the agent to Agent Engine using inline source deployment.

The key difference from default deployment is using `AgentEngineApp.from_agent()` which packages the source code directly.

In [ ]:
from vertexai import agent_engines
from vertexai.agent_engines import AgentEngineApp

# Create the app from the agent (inline source)
app = AgentEngineApp.from_agent(
    agent=agent,
    enable_tracing=True,
)

print("App created successfully")

### Deploy to Agent Engine

In [ ]:
# Deploy the app
# This may take a few minutes
remote_app = agent_engines.create(
    app,
    requirements=REQUIREMENTS,
    display_name="time-agent-inline",
)

print(f"Deployed agent: {remote_app.resource_name}")

## Test the Deployed Agent

Now let's test the deployed agent by sending it a query.

In [ ]:
# Create a session with the remote agent
remote_session = remote_app.create_session(user_id="test_user")

print(f"Created session: {remote_session['id']}")

In [ ]:
# Query the agent
response = remote_app.stream_query(
    user_id="test_user",
    session_id=remote_session["id"],
    message="What time is it in Tokyo?",
)

# Print the response
for chunk in response:
    if hasattr(chunk, "content") and chunk.content:
        for part in chunk.content.parts:
            if hasattr(part, "text") and part.text:
                print(part.text, end="")

print()  # Newline at end

## Cleaning up

Delete the deployed agent to avoid ongoing charges.

In [ ]:
# Delete the deployed agent
delete_agent = False  # @param {type:"boolean"}

if delete_agent:
    remote_app.delete()
    print("Agent deleted successfully")

In [ ]:
# Delete the staging bucket
delete_bucket = False  # @param {type:"boolean"}

if delete_bucket:
    !gsutil rm -r {STAGING_BUCKET}
    print("Bucket deleted successfully")

## Summary

In this notebook, you learned how to:

1. Create an ADK agent with tools
2. Test the agent locally
3. Deploy to Agent Engine using inline source with `AgentEngineApp.from_agent()`
4. Query the deployed agent

### Next steps

- Learn more about [ADK](https://google.github.io/adk-docs/)
- Explore [Agent Engine documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)
- Try adding more complex tools and multi-agent orchestration